## Roll Rate Analysis

En esta sección, se va a utilizar *Roll Rate Analysis* con el fin de poder determinar de una manera más acorde a los datos, aquellos clientes que se pueden identificar como buenos o malos, dependiendo de cuántos días de atraso se registran en el pago de sus cuentas.

Para efectos prácticos, se va a tomar como puntos de análisis, el último mes y el mes anterior, también se va a ignorar a los clientes que para esa fecha no cuentan con un préstamo registrado.